In [1]:
# Imports

import json


In [5]:
f = open("../../data/News_Category_Dataset_v2.json", "r")
Line = f.readline()
js = json.loads(Line)

print(js["headline"])

There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV
